# Sentiment Analysis on Twitter: Deep Learning Approach 1
Author: Niloy Farhan

Model Description:

Embedding Layer -> Spatial Dropout 1d -> Bidirectional LSTM Layer -> Dense(300) Layer -> Softmax Activation Layer

Best Score after pre-processing (Own Embedding):

72% Accuracy

# Imports

In [1]:
import numpy as np 
import pandas as pd 
import re
import string
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras import layers
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, GRU
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from sklearn.utils import resample
import nltk
from nltk.corpus import stopwords


In [2]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# TPU INITIALIZATION

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.8.0
Running on TPU  ['10.86.121.202:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.86.121.202:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.86.121.202:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


# Dataset Loading

In [4]:
df1 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2013dev-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df1 = df1.drop(labels=["id"], axis = 1)
df2 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2013test-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df2 = df2.drop(labels=["id"], axis = 1)
df3 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2013train-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df3 = df3.drop(labels=["id"], axis = 1)
df4 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2014sarcasm-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df4 = df4.drop(labels=["id"], axis = 1)
df5 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2015test-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df5 = df5.drop(labels=["id"], axis = 1)
df6 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2015train-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df6 = df6.drop(labels=["id"], axis = 1)
df7 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2016dev-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df7 = df7.drop(labels=["id"], axis = 1)
df8 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2016devtest-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df8 = df8.drop(labels=["id"], axis = 1)
df9 = pd.read_csv("/content/drive/MyDrive/Bracu/Spring2022/CSE440/Project/Dataset/twitter-2016train-A.txt", sep = '\t', names = ['id','sentiment','tweets'])
df9 = df9.drop(labels=["id"], axis = 1)

In [6]:
# df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9]).reset_index(drop=True)
df = pd.concat([df3,df5,df8,df9]).reset_index(drop=True)

df

,sentiment,tweets
0,positive,Gas by my house hit $3.39!!!! I\u2019m going t...
1,negative,Theo Walcott is still shit\u002c watch Rafa an...
2,negative,its not that I\u2019m a GSP fan\u002c i just h...
3,negative,Iranian general says Israel\u2019s Iron Dome c...
4,neutral,Tehran\u002c Mon Amour: Obama Tried to Establi...
...,...,...
19937,positive,@Racalto_SK ok good to know. Punting at MetLif...
19938,neutral,everyone who sat around me at metlife was so a...
19939,neutral,what giants or niners fans would wanna go to t...
19940,positive,Anybody want a ticket for tomorrow Colombia vs...


# Balancing the dataset

Our dataset is unbalanced when it comes to negative sentiment. So, we will resample to balance it.

In [7]:
df['sentiment'].value_counts()

positive    8689
neutral     8255
negative    2998
Name: sentiment, dtype: int64

In [8]:
def dataset_balance(data):
    data_positive = data[(data['sentiment']=='positive')]
    data_neutral = data[(data['sentiment']=='neutral')]
    data_negative = data[(data['sentiment']=='negative')]
    # data['sentiment'].value_counts()[2]
    # data_positive_downsampled = resample(data_positive, replace = True, n_samples = 6000, random_state = 42)
    # data_neutral_downsampled = resample(data_neutral, replace = True, n_samples =  data['sentiment'].value_counts()[0], random_state = 42)
    data_negative_upsampled = resample(data_negative, replace = True, n_samples =  data['sentiment'].value_counts()[1], random_state = 42)
    balanced_dataset = pd.concat([data_positive,data_neutral,data_negative_upsampled]).reset_index(drop=True)
    return balanced_dataset

In [9]:
df = dataset_balance(df)
df['sentiment'].value_counts()

positive    8689
neutral     8255
negative    8255
Name: sentiment, dtype: int64

# PRE-PROCESSING

1. removed link
2. removed linebreaks
3. removed extra spaces
4. removed punctuation
5. removed stopwords
6. lowercased the tweet
7. extracted hashtags and added at the end of the sentence
8. extracted mentions and added at the end of the sentence

In [10]:
def label_encoder(x):
    if x == 'positive': return 2
    elif x == 'negative': return 0
    else: return 1

def clean_text(x):
    x = re.sub(r'https?://\S+', '', x) 
    x = re.sub(r'#\w+', '', x) 
    x = re.sub(r'@\w+', '', x) 
    x = re.sub(r'\n',' ',x) 
    x = re.sub('\s+', ' ', x).strip()
    x = re.sub('\.','',x) 
    for p in string.punctuation:
        x = re.sub('\{}'.format(p),'',x)
    return x.lower()
def find_hashtags(tweet):
    return " ".join([match.group(0)[1:] for match in re.finditer(r"#\w+", tweet)]) or ''

def find_mentions(tweet):
    return " ".join([match.group(0)[1:] for match in re.finditer(r"@\w+", tweet)]) or ''

def pre_process_text(df):
    stop = stopwords.words('english')
    df['clean_tweet'] = df['tweets'].apply(lambda x: clean_text(x))
    df['hashtags'] = df['tweets'].apply(lambda x: find_hashtags(x))
    df['mentions'] = df['tweets'].apply(lambda x: find_mentions(x))
    df['sentiment'] = df['sentiment'].apply(lambda x: label_encoder(x))
    df['clean_tweet_without_stopword'] = df['clean_tweet'].apply(lambda x:' '.join([word for word in x.split() if word not in stop]))
    df['final'] = df['clean_tweet_without_stopword']+" "+df['hashtags']+" "+df['mentions']
    # df['clean_tweet']+" "+
    return df


In [11]:
final_dataset = pre_process_text(df)

In [12]:
final_dataset.tail()

,sentiment,tweets,clean_tweet,hashtags,mentions,clean_tweet_without_stopword,final
25194,0,One of my Magic Mike XXL co-workers and the co...,one of my magic mike xxl coworkers and the coo...,,,one magic mike xxl coworkers coordinator premi...,one magic mike xxl coworkers coordinator premi...
25195,0,Sreven Taylor will look pretty daft after the ...,sreven taylor will look pretty daft after the ...,,,sreven taylor look pretty daft mackems win tom...,sreven taylor look pretty daft mackems win tom...
25196,0,i hope justin's concert thursday gets cancelle...,i hope justins concert thursday gets cancelled...,,,hope justins concert thursday gets cancelled b...,hope justins concert thursday gets cancelled b...
25197,0,About as much interest in today as I do in the...,about as much interest in today as i do in the...,zero,,much interest today rugby world cup,much interest today rugby world cup zero
25198,0,@Becker_Boris It\u2019s almost 2 am in BKK but...,itu2019s almost 2 am in bkk but i canu2019t sl...,,Becker_Boris,itu2019s almost 2 bkk canu2019t sleep game isn...,itu2019s almost 2 bkk canu2019t sleep game isn...


Longest sequence

In [13]:
final_dataset['final'].apply(lambda x:len(str(x).split())).max()

689

In [14]:
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 750
EMBEDDING_DIM = 200

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(final_dataset['final'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 38284 unique tokens.


In [16]:
X = tokenizer.texts_to_sequences(final_dataset['final'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (25199, 750)


# Data spliting

In [17]:
Y = pd.get_dummies(final_dataset['sentiment']).values
print('Shape of label tensor:', Y.shape)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

Shape of label tensor: (25199, 3)
(22679, 750) (22679, 3)
(2520, 750) (2520, 3)


# My Model

In [22]:
with tpu_strategy.scope():
  model = Sequential()
  model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
  model.add(SpatialDropout1D(0.2))
  model.add(Bidirectional(LSTM(300, dropout=0.2, recurrent_dropout=0.2)))
  model.add(Dense(300, activation='relu'))
  model.add(Dense(3, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 750, 200)          10000000  
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 750, 200)         0         
 lDropout1D)                                                     
                                                                 
 bidirectional_2 (Bidirectio  (None, 600)              1202400   
 nal)                                                            
                                                                 
 dense_4 (Dense)             (None, 300)               180300    
                                                                 
 dense_5 (Dense)             (None, 3)                 903       
                                                                 
Total params: 11,383,603
Trainable params: 11,383,603


In [23]:
epochs = 25
batch_size = 32*tpu_strategy.num_replicas_in_sync
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=2, min_delta=0.0001)])

Epoch 1/25
80/80 [==============================] - 107s 1s/step - loss: 0.9191 - accuracy: 0.5405 - val_loss: 0.7119 - val_accuracy: 0.6812
Epoch 2/25
80/80 [==============================] - 79s 987ms/step - loss: 0.4656 - accuracy: 0.8116 - val_loss: 0.6207 - val_accuracy: 0.7315
Epoch 3/25
80/80 [==============================] - 79s 985ms/step - loss: 0.2185 - accuracy: 0.9184 - val_loss: 0.6996 - val_accuracy: 0.7478
Epoch 4/25
80/80 [==============================] - 79s 986ms/step - loss: 0.1121 - accuracy: 0.9613 - val_loss: 0.8139 - val_accuracy: 0.7434


# Test Data score

In [24]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

79/79 [==============================] - 8s 86ms/step - loss: 0.9239 - accuracy: 0.7187
Test set
  Loss: 0.924
  Accuracy: 0.719
